# Assignment: Connect Multiple Tools via APIs and Enable Multi-Agent Triggers


---

### Objective:
This assignment challenges you to build a sophisticated **multi-agent system** that integrates with **multiple external APIs (tools)**. The key aspect is to design the agents' collaboration such that the output of one agent or the successful execution of one tool **triggers** another agent to perform a subsequent action, potentially involving another tool. You will orchestrate a workflow where agents dynamically decide which tools to use and when, based on the conversation flow and task requirements.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API (e.g., OpenAI's GPT-4o, GPT-4, GPT-3.5-turbo). Configure your API key securely.
2.  **Multi-Agent Framework**: **AutoGen** is highly recommended for its powerful multi-agent orchestration capabilities. Alternatively, you can use `CrewAI` or build a custom agent system, but AutoGen will simplify tool integration and agent communication.
3.  **External APIs (Tools)**: You must integrate with at least **two distinct external APIs** that provide data or actions. Good combinations could be:
    * **Weather API** (e.g., OpenWeatherMap, WeatherAPI.com) - to get current weather.
    * **News API** (e.g., NewsAPI.org, GNews API) - to get trending news.
    * **Public REST API for Jokes/Facts** (e.g., Chuck Norris Jokes API, Numbers API) - for simple data retrieval.
    * **Stock Market Data API** (e.g., Alpha Vantage, Twelve Data) - for stock prices.
    * *(Avoid APIs requiring complex OAuth for this assignment's scope)*.
4.  **Scenario: Event Planner Assistant**: Design a multi-agent system that helps plan a simple event based on a user's query. The workflow should demonstrate multi-agent triggers. For example:
    * **User asks**: "I want to plan an outdoor event in London this weekend. What's the weather like and what's happening in the news?"
    * **Planner Agent**: Recognizes the need for weather and news.
    * **Weather Agent**: Gets weather for London.
    * **News Agent**: Gets news for London or related to events.
    * **Aggregator/Reporter Agent**: Combines info and provides recommendations (e.g., "Weather is sunny, good for outdoor. No major events in news. Consider [event idea].")
    * **Trigger Example**: The Planner agent triggers the Weather Agent, whose successful output might then trigger the News Agent.
5.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
6.  **Analysis**: Explain your agent design, tool integration, and how multi-agent triggers were achieved. Discuss the advantages and challenges.

---

## Part 1: Setup and API Configuration
Configure your LLM and external API keys. Install necessary libraries.

### Task 1.1: Install Libraries and Configure API Keys
Install `pyautogen`, `python-dotenv`, and `requests`. Set up your LLM and external API keys.

For external APIs, you'll need:
* **OpenWeatherMap API Key**: Get from [openweathermap.org/api](https://openweathermap.org/api)
* **NewsAPI API Key**: Get from [newsapi.org](https://newsapi.org/)

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install pyautogen python-dotenv requests --quiet

import autogen
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- IMPORTANT: Create a .env file in the same directory as this notebook with the following lines: ---
# OPENAI_API_KEY="YOUR_OPENAI_API_KEY_HERE"
# OPENWEATHERMAP_API_KEY="YOUR_OPENWEATHERMAP_API_KEY_HERE"
# NEWSAPI_API_KEY="YOUR_NEWSAPI_API_KEY_HERE"

# Configure LLM (OpenAI model recommended)
llm_config = {
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-4o", "gpt-4", "gpt-3.5-turbo"], # Prioritize capable models
        },
    ),
    "temperature": 0.2, # Lower temperature for factual and analytical tasks
}

# Fallback if OAI_CONFIG_LIST not found or for direct API key usage
if not llm_config["config_list"] and os.getenv("OPENAI_API_KEY"):
    print("Using OPENAI_API_KEY from environment variable as fallback for LLM config.")
    llm_config["config_list"] = [
        {"model": os.getenv("OPENAI_MODEL_ID", "gpt-4o"), "api_key": os.getenv("OPENAI_API_KEY")}
    ]
elif not llm_config["config_list"]:
    print("WARNING: No LLM configuration found. Please set OPENAI_API_KEY or create OAI_CONFIG_LIST.")

# Get external API keys
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")
NEWSAPI_API_KEY = os.getenv("NEWSAPI_API_KEY")

if not OPENWEATHERMAP_API_KEY:
    print("WARNING: OPENWEATHERMAP_API_KEY not found.")
if not NEWSAPI_API_KEY:
    print("WARNING: NEWSAPI_API_KEY not found.")

print("AutoGen environment and API keys configured!")

---

## Part 2: Define Tools (API Wrappers)
Create Python functions that encapsulate API calls to your chosen external services. These functions will be callable by your AutoGen agents.

### Task 2.1: `get_current_weather` Tool
Create a function that takes `location` (city, country) and `unit` (celsius/fahrenheit) and calls the OpenWeatherMap API. Handle potential errors gracefully.

In [ ]:
def get_current_weather(location: str, unit: str = "celsius") -> str:
    """
    Gets the current weather for a specified location.
    :param location: The city and country (e.g., 'London, UK') to get weather for.
    :param unit: Temperature unit: 'celsius' or 'fahrenheit'. Defaults to celsius.
    :return: A string summarizing the weather or an error message.
    """
    if not OPENWEATHERMAP_API_KEY:
        return "Weather API key not configured."

    base_url = "https://api.openweathermap.org/data/2.5/weather"
    units_param = "metric" if unit.lower() == "celsius" else "imperial"

    params = {
        "q": location,
        "appid": OPENWEATHERMAP_API_KEY,
        "units": units_param
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        data = response.json()

        if data.get("cod") == "404":
            return f"Weather for '{location}' not found."

        main = data.get("main", {})
        weather_desc = data.get("weather", [{}])[0].get("description", "")
        temp = main.get("temp")
        humidity = main.get("humidity")

        if temp is not None:
            return f"Current weather in {location}: {temp}°{unit.upper()[0]}, {weather_desc}, humidity: {humidity}%."
        else:
            return f"Could not retrieve temperature for {location}."
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather for {location}: {e}"
    except Exception as e:
        return f"An unexpected error occurred while processing weather data: {e}"

print("get_current_weather tool defined!")

### Task 2.2: `get_latest_news` Tool
Create a function that takes a `topic` and `location` (optional) and calls the NewsAPI. Return a summary of news articles. Handle errors.

In [ ]:
def get_latest_news(topic: str = "general", location: str = "", count: int = 3) -> str:
    """
    Gets the latest news articles for a specified topic and optional location.
    :param topic: The news topic or keyword to search for (e.g., 'technology', 'events').
    :param location: Optional. The location to filter news by (e.g., 'London', 'UK'). NewsAPI typically uses country codes.
    :param count: The number of news articles to retrieve. Defaults to 3.
    :return: A formatted string listing the news articles or an error message.
    """
    if not NEWSAPI_API_KEY:
        return "News API key not configured."

    base_url = "https://newsapi.org/v2/everything" # Use 'everything' for topic-based search
    if count > 5: # Limit to avoid excessive API calls on free tier
        count = 5

    params = {
        "q": topic, # Primary query parameter
        "apiKey": NEWSAPI_API_KEY,
        "pageSize": count,
        "language": "en" # Limit to English news
    }

    # NewsAPI 'country' parameter is for 'top-headlines', not 'everything' search
    # For location relevance in 'everything', the LLM will integrate 'location' into 'q' if needed.
    # If you want strictly country-specific news, you'd use 'top-headlines' and ISO 3166-1 alpha-2 country codes.
    # For simplicity, we'll let the LLM integrate location into the query if relevant for 'everything' endpoint.
    if location:
        params["q"] = f"{topic} {location}"

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        articles = data.get("articles", [])
        if not articles:
            return f"No news found for topic '{topic}' in '{location}'."

        news_summary = []
        for i, article in enumerate(articles):
            title = article.get("title", "No Title")
            source = article.get("source", {}).get("name", "Unknown Source")
            url = article.get("url", "")
            news_summary.append(f"{i+1}. {title} (Source: {source}) - {url}")

        return f"Latest news on '{topic}' in '{location}':\n" + "\n".join(news_summary)
    except requests.exceptions.RequestException as e:
        return f"Error fetching news for '{topic}' in '{location}': {e}"
    except Exception as e:
        return f"An unexpected error occurred while processing news data: {e}"

print("get_latest_news tool defined!")

---

## Part 3: Define Agents and their Capabilities
Create different `AssistantAgent`s with specific roles and register the tools to the appropriate agents.

### Task 3.1: `UserProxyAgent` (The Initiator & Executor)
This agent initiates the conversation, executes tool calls when delegated, and manages termination.

In [ ]:
def is_termination_message(message):
    return "TERMINATE" in message.get("content", "").upper() or "FINISH" in message.get("content", "").upper()

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER", # Set to NEVER for full automation
    max_consecutive_auto_reply=15, # Allow enough turns for complex tasks
    is_termination_msg=is_termination_message,
    code_execution_config={
        "work_dir": "event_planner_workdir", # Directory for code execution
        "use_docker": False, # Set to True if you have Docker installed for isolated execution
    },
    system_message="A human administrator who oversees the event planning team. Provide high-level event planning requests and approve final recommendations. Conclude with 'TERMINATE' when satisfied."
)

print("UserProxyAgent 'Admin' created!")

### Task 3.2: Specialized Agents (`WeatherAgent`, `NewsAgent`, `EventPlannerAgent`)
Define `AssistantAgent`s for each role. Assign tools to the `UserProxyAgent` but make them callable by specific `AssistantAgent`s.

In [ ]:
# Weather Agent
weather_agent = autogen.AssistantAgent(
    name="WeatherAgent",
    system_message=(
        "You are a specialized weather information retriever. Your task is to use the 'get_current_weather' tool "
        "to find weather conditions for a given location and unit. Report the findings clearly. "
        "Only provide weather information and pass it to the EventPlannerAgent."
    ),
    llm_config=llm_config,
)

# News Agent
news_agent = autogen.AssistantAgent(
    name="NewsAgent",
    system_message=(
        "You are a specialized news researcher. Your task is to use the 'get_latest_news' tool "
        "to find relevant news articles for a given topic and location. Summarize the key headlines. "
        "Only provide news information and pass it to the EventPlannerAgent."
    ),
    llm_config=llm_config,
)

# Event Planner Agent (Orchestrator and Reporter)
event_planner_agent = autogen.AssistantAgent(
    name="EventPlannerAgent",
    system_message=(
        "You are a central event planning assistant. Your role is to understand the user's event request, "
        "coordinate with the WeatherAgent and NewsAgent to gather necessary information, "
        "and then synthesize all gathered data into a comprehensive event recommendation. "
        "Suggest suitable outdoor event ideas based on weather and news. Once the recommendation is ready, "
        "present it clearly to the Admin. Conclude your final message to Admin with 'TERMINATE'."
    ),
    llm_config=llm_config,
)

print("Specialized agents created!")

### Task 3.3: Register Tools
Register the tools to the `UserProxyAgent`, making them available for the other agents to call via the `UserProxyAgent`.

In [ ]:
# Register weather tool
user_proxy.register_for_execution(get_current_weather, caller=weather_agent)
weather_agent.register_for_tool_use(user_proxy, get_current_weather)

# Register news tool
user_proxy.register_for_execution(get_latest_news, caller=news_agent)
news_agent.register_for_tool_use(user_proxy, get_latest_news)

print("Tools registered for execution and use by agents!")

---

## Part 4: Orchestrate Multi-Agent Triggers and Conversation Flow
Define the `GroupChat` and `GroupChatManager` to enable dynamic agent interaction and tool triggering.

### Task 4.1: Create `GroupChat` and `GroupChatManager`
Assemble your agents into a `GroupChat` and manage their conversation with a `GroupChatManager`.

In [ ]:
agents = [user_proxy, weather_agent, news_agent, event_planner_agent]

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=25, # Allow enough turns for complex tasks
    speaker_selection_method="auto", # AutoGen decides who speaks next
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

print("GroupChat and GroupChatManager created!")

### Task 4.2: Initiate the Multi-Agent Conversation
Start the chat with a specific event planning query that requires both weather and news information.

In [ ]:
# Make sure the work directory exists
if not os.path.exists(user_proxy.code_execution_config["work_dir"]):
    os.makedirs(user_proxy.code_execution_config["work_dir"])

event_query = "I need to plan an outdoor picnic event in London this Saturday. What's the weather forecast, and are there any major local news or events happening that might affect it? Provide me with a recommendation."

print("\n--- Initiating Event Planning Conversation ---")

chat_history = user_proxy.initiate_chat(
    manager, # The manager agent to start the group chat
    message=event_query,
    clear_history=True, # Clear previous chat history
    silent=False, # Set to True to suppress console output during chat
)

print("\n--- Event Planning Conversation Ended ---")

---

## Part 5: Analysis and Reflection
Review the console output and the final event recommendation. Answer the following questions.

### Task 5.1: Multi-Agent Triggers and Workflow
* **Trigger Mechanism**: Describe how the initial query from `Admin` implicitly or explicitly triggered the `EventPlannerAgent` to then trigger `WeatherAgent` and `NewsAgent`. Analyze the conversation flow to show this sequence.
* **Information Flow**: Trace the flow of information: how did the weather data and news data get from the respective tool functions to the `EventPlannerAgent` for synthesis?
* **Collaboration**: How effectively did the agents collaborate? Did they communicate clearly and pass necessary context? Provide examples from the chat log.

### Task 5.2: Tool Integration and Orchestration
* **Tool Use**: Did the `WeatherAgent` and `NewsAgent` correctly identify when to use their respective tools and with appropriate parameters? Provide console snippets showing tool calls and their results.
* **LLM Role**: How crucial was the LLM in the `AssistantAgent`s for deciding *when* and *how* to use the tools, as opposed to rigid, hardcoded logic? What benefits does this dynamic decision-making provide?
* **Error Handling**: How did the system handle cases where an API call failed (e.g., if an API key was invalid or a network error occurred)? Did it recover gracefully or fail?

### Task 5.3: Limitations and Future Enhancements
* **Current Limitations**: What are the current limitations of your multi-agent event planner? (e.g., lack of real-time event data, handling complex follow-up questions, reliance on exact location names).
* **Scalability**: How would you enhance this system for a more robust and scalable event planning service? Consider:
    * Adding more specialized agents (e.g., a "Venue Booker", "Catering Agent", "Budget Agent").
    * Integrating more complex APIs (e.g., calendar APIs, ticketing APIs, real-time event databases).
    * Implementing mechanisms for human feedback and correction within the loop.
* **Robustness**: How could you make the system more robust against ambiguous queries or unexpected API responses?

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Make sure your `.env` file (or equivalent API key setup) is mentioned but **NOT** included in the submitted notebook for security reasons.
* Save your Jupyter Notebook as `[YourName]_Multi_Agent_Tool_Orchestration_Assignment.ipynb`.